In [1]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F #take a long time 

d:\mini\envs\pine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [121]:
df_1_review = pd.read_csv('โซน หลักหก\df_review_combined.csv')
df_2_review = pd.read_csv('โซน รังสิต\df_review_combined3.csv')
df_3_review = pd.read_csv('โซน คลองหนึ่ง\df_review_combined2.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\eieiz\AppData\Local\Temp\ipykernel_10360\2527533981.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_1_review = pd.read_csv('โซน หลักหก\df_review_combined.csv')
C:\Users\eieiz\AppData\Local\Temp\ipykernel_10360\2527533981.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_2_review = pd.read_csv('โซน รังสิต\df_review_combined3.csv')
C:\Users\eieiz\AppData\Local\Temp\ipykernel_10360\2527533981.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_3_review = pd.read_csv('โซน คลองหนึ่ง\df_review_combined2.csv')


https://huggingface.co/poom-sci/WangchanBERTa-finetuned-sentiment

In [107]:
import numpy as np

In [102]:
from transformers import pipeline

In [104]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "poom-sci/WangchanBERTa-finetuned-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)




Device set to use cuda:0


In [ ]:
classifier('dddddddddddddddddddddddd', truncation=True)

[{'label': 'neu', 'score': 0.803070068359375}]

In [108]:
df_1_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_review_main  824 non-null    object 
 1   review_rating   824 non-null    float64
 2   review_text     824 non-null    object 
dtypes: float64(1), object(2)
memory usage: 20.0+ KB


In [122]:
df_1_review['review_text'] = df_1_review['review_text'].astype(str)
df_2_review['review_text'] = df_2_review['review_text'].astype(str)
df_3_review['review_text'] = df_3_review['review_text'].astype(str)


In [123]:
for i in df_1_review['review_text']:
    if i == 'nan':
        print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [124]:
df_1_review['review_text'] = df_1_review['review_text'].replace('nagative', 'negative')
df_2_review['review_text'] = df_2_review['review_text'].replace('nagative', 'negative')
df_3_review['review_text'] = df_3_review['review_text'].replace('nagative', 'negative')


In [125]:
df_1_review[df_1_review['review_text'] == 'negative']

,id_review_main,review_rating,review_text
309,review_4,2.0,negative
620,review_14,1.0,negative
841,review_47,2.0,negative


In [126]:
from tqdm import tqdm

def apply_sentiment(df, pipe):
    sentiments = []
    for text in tqdm(df['review_text'], desc="Processing Sentiment"):
        text = str(text).strip()  # ✅ แปลงเป็น string และลบช่องว่าง

        if text.lower() == 'nan':  # ✅ ถ้าเดิมเป็น NaN
            sentiments.append(np.nan)
        elif text == 'positive':
            sentiments.append('pos')
        elif text == 'negative':
            sentiments.append('neg')
        else:
            try:
                result = pipe(text, truncation=True)[0]['label']
                sentiments.append(result)
            except Exception as e:
                result = pipe(text[:300], truncation=True)[0]['label']
                sentiments.append(result)
    df['sentiment'] = sentiments
    return df


In [127]:
df_1_review = apply_sentiment(df_1_review, classifier)
df_2_review = apply_sentiment(df_2_review, classifier)
df_3_review = apply_sentiment(df_3_review, classifier)

Processing Sentiment: 100%|██████████| 629/629 [00:05<00:00, 121.67it/s]


In [128]:
df_1_review

,id_review_main,review_rating,review_text,sentiment
0,review_1,3.0,เด็กเสิร์ฟน้อยขนาดไปวันเสาร์นั่งกินอาหารตั้งแต...,neg
1,review_1,5.0,มาครั้งแรกอาหารอร่อยถูกปาก.บรรยากาศดี.กาแฟอร่อ...,pos
2,review_1,4.0,บรรยากาศดีค่ะ ถ้ามาช่วงค่ำมีดนตรี อาหาร น่าจ...,pos
3,review_1,5.0,ร้านสวยยยยมากกก บรรยากาศดี ตอนแรกไม่ได้ตั้งใจแ...,pos
4,review_1,4.0,Theme Cafe & Bistro คาเฟ่โรงนาสีแดงสไตล์ยุโรปต...,pos
...,...,...,...,...
844,review_47,5.0,positive,pos
845,review_47,5.0,positive,pos
846,review_49,5.0,กาแฟและเครื่องดื่มอร่อย ของคุณภาพดี แต่ราคาน่า...,pos
847,review_49,5.0,positive,pos


In [129]:
df_1_review.to_csv('df_1_review_clean.csv', index=False,encoding='utf-8-sig')
df_2_review.to_csv('df_2_review_clean.csv', index=False,encoding='utf-8-sig')
df_3_review.to_csv('df_3_review_clean.csv', index=False,encoding='utf-8-sig')